In [1]:
import tensorflow as tf
import numpy as np

## sharing parameters in mulitple components in tf - by method attribute

In [2]:
tf.reset_default_graph()

def relu(x):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = (int(x.get_shape()[1]), 1)
        w = tf.Variable(tf.truncated_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(x, w), b, name="z")
        return tf.maximum(z, relu.threshold, name="max")
    
X = tf.placeholder(tf.float32, shape=(None, 4), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [8]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    print(output.eval(feed_dict={X: np.random.rand(32, 4).astype(np.float32)}))

[[ 1.81776404]
 [ 2.12920499]
 [ 1.23818207]
 [ 1.7762363 ]
 [ 1.61449015]
 [ 1.21516371]
 [ 1.59917831]
 [ 0.3899439 ]
 [ 1.30287027]
 [ 1.16305327]
 [ 1.12576187]
 [ 0.57863039]
 [ 1.24174798]
 [ 1.01707304]
 [ 0.86130184]
 [ 0.        ]
 [ 1.03433323]
 [ 0.57876635]
 [ 0.94072402]
 [ 2.18578339]
 [ 1.95505798]
 [ 0.34956798]
 [ 1.77161562]
 [ 0.15981589]
 [ 0.26153257]
 [ 0.30289859]
 [ 0.40174085]
 [ 0.5301947 ]
 [ 1.42607319]
 [ 0.97118437]
 [ 2.33050036]
 [ 0.56121892]]


## sharing variables in multiple components by dynamically creating on-the-fly variables

In [19]:
tf.reset_default_graph()

with tf.variable_scope("test"):
    w = tf.get_variable("w", shape=[], initializer=tf.constant_initializer(5))
    print(w.name)
    tf.get_variable_scope().reuse_variables()
    w1 = tf.get_variable("w")
    print(w1.name)
    print(w == w1)

test/w:0
test/w:0
True


In [35]:
## difference between tf.variable_scope and tf.names_scope
## tf.namescope will be ignored by tf.get_variable
tf.reset_default_graph()

with tf.name_scope("b1"):
    w1 = tf.get_variable("w1", shape=[], initializer=tf.constant_initializer(0.0))
#     w3 = tf.get_variable("w1", shape=[], initializer=tf.constant_initializer(0.0))
    w2 = tf.Variable(0.0, name="w2")

print(w1.name, w2.name)

w1:0 b1/w2:0


In [37]:
tf.reset_default_graph()

with tf.variable_scope("b2", reuse=False):
    w1 = tf.get_variable("w1", shape=[], initializer=tf.constant_initializer(0.0))
    w2 = tf.Variable(0.0, name="w2")
    
print(w1.name, w2.name)

b2/w1:0 b2/w2:0


In [25]:
tf.reset_default_graph()

def relu(x):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = [int(x.get_shape()[1]), 1]
        w = tf.Variable(tf.truncated_normal(w_shape), name="w")
        b = tf.Variable(0.0, name="b")
        z = tf.add(tf.matmul(x, w), b, name="z")
        return tf.maximum(z, threshold, name="max")
    
X = tf.placeholder(tf.float32, shape=(None, 4), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", [], tf.float32, tf.constant_initializer(0.0))
relus = [relu(X) for _ in range(5)]
output = tf.add_n(relus, name="output")

In [27]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    print(output.eval(feed_dict={X: np.random.rand(32, 4)}).shape)

(32, 1)


In [38]:
sess.close()